- https://www.kaggle.com/code/tobyanderson/health-survey-analysis
- https://www.kaggle.com/code/gopalkholade/diabetes-prediction
- https://www.kaggle.com/code/sohamohajeri/nhanes-dimensionality-reduction-and-clustering 
- https://www.kaggle.com/code/lcchennn/stroke-prediction-using-machine-learning
- https://www.kaggle.com/code/kaggleaar/diseases-and-medications-analysis

TODO:
1. Featurize the dataset. Clean, organize, and transform the dataset into a useable format. Drop columns, fix null values, etc
2. Build functions `dataLoader(), logger(), cluster(), dimRed()`
3. Extract metadata, cluster, dimension reduction
4. Attempt to answer business questions. 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Load and Join Data

In [3]:
labs = pd.read_csv('nhnes/labs.csv')
exam = pd.read_csv('nhnes/examination.csv')
demo = pd.read_csv('nhnes/demographic.csv')
diet = pd.read_csv('nhnes/diet.csv')
ques = pd.read_csv('nhnes/questionnaire.csv')

In [34]:
dfs = [labs, exam, demo, diet, ques]
dfs_str = ["Labs", "Exam", "Demo", "Diet", "Ques"]
dfs_lens = len(dfs)

# Find any shared columns in all tables
for i in range(dfs_lens):
    for j in range(i+1, dfs_lens):
        print("{} and {} datasets share ".format(dfs_str[i], dfs_str[j]), end=" ")
        print(np.intersect1d(dfs[i].columns, dfs[j].columns), "column(s)")

print()

# Shape of each table
for i in range(dfs_lens):
    print("Shape of {}: {}".format(dfs_str[i], dfs[i].shape))

Labs and Exam datasets share  ['SEQN'] column(s)
Labs and Demo datasets share  ['SEQN'] column(s)
Labs and Diet datasets share  ['SEQN'] column(s)
Labs and Ques datasets share  ['SEQN'] column(s)
Exam and Demo datasets share  ['SEQN'] column(s)
Exam and Diet datasets share  ['SEQN'] column(s)
Exam and Ques datasets share  ['SEQN'] column(s)
Demo and Diet datasets share  ['SEQN'] column(s)
Demo and Ques datasets share  ['SEQN'] column(s)
Diet and Ques datasets share  ['SEQN'] column(s)

Shape of Labs: (9813, 424)
Shape of Exam: (9813, 224)
Shape of Demo: (10175, 47)
Shape of Diet: (9813, 168)
Shape of Ques: (10175, 953)


In [35]:
# Join on shared column
df = labs.merge(exam, on="SEQN", how = "outer")
df = df.merge(demo, on="SEQN", how="outer")
df = df.merge(diet, on="SEQN", how="outer")
df = df.merge(ques, on="SEQN", how="outer")
print(df.shape)


(10175, 1812)


In [41]:
df

,SEQN,URXUMA,URXUMS,URXUCR.x,URXCRS,URDACT,WTSAF2YR.x,LBXAPB,LBDAPBSI,LBXSAL,...,WHD080U,WHD080L,WHD110,WHD120,WHD130,WHD140,WHQ150,WHQ030M,WHQ500,WHQ520
0,73557,4.3,4.3,39.0,3447.6,11.03,NaN,NaN,NaN,4.1,...,NaN,40.0,270.0,200.0,69.0,270.0,62.0,NaN,NaN,NaN
1,73558,153.0,153.0,50.0,4420.0,306.00,NaN,NaN,NaN,4.7,...,NaN,NaN,240.0,250.0,72.0,250.0,25.0,NaN,NaN,NaN
2,73559,11.9,11.9,113.0,9989.2,10.53,142196.890197,57.0,0.57,3.7,...,NaN,NaN,180.0,190.0,70.0,228.0,35.0,NaN,NaN,NaN
3,73560,16.0,16.0,76.0,6718.4,21.05,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,3.0
4,73561,255.0,255.0,147.0,12994.8,173.47,142266.006548,92.0,0.92,4.3,...,NaN,NaN,150.0,135.0,67.0,170.0,60.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10170,83551,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,230.0,NaN,265.0,23.0,NaN,NaN,NaN
10171,83593,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,175.0,150.0,NaN,210.0,21.0,NaN,NaN,NaN
10172,83635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10173,83652,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,230.0,174.0,NaN,283.0,39.0,NaN,NaN,NaN
